# KNU 감성사전을 통한 단어의 긍정/부정을 판단하기

In [1]:
# ### KNU 한국어 감성사전 -DI LAB
# - SentiWord info.json 한국어 감성사전 제공

import json
import pandas as pd
from tqdm import tqdm

In [2]:
SentiWord = pd.read_json('SentiWord_info.json') # json 파일 불러옴
SentiWord.to_csv('SentiWord_info.csv', index=False) # json 파일을 csv로 저장

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect("practice.db") # practice.db 연결
c = conn.cursor() # db를 사용하기 위한 cursor

In [5]:
c.execute("SELECT text FROM newspapers") # newspaper table의 내용을 추출

articles = [] # list 선언

for row in c.fetchall():  # table의 내용을 반복하여 추출
  articles.append(row)  # 해당 내용을 articles list에 넣음

print(articles[0])  # articles[i]가 title, text 등 모든 내용을 포함한 기사 1개

('로또복권 운영사 동행복권은 제974회 로또복권 추첨에서 ‘1, 2, 11, 16, 39, 44’이 1등 당첨번호로 뽑혔다고 31일 밝혔다. 2등 보너스 번호는 ‘32’다.당첨번호 6개를 모두 맞힌 1등 당첨자는 17명으로 13억1703만4523원씩 받는다. 당첨번호 5개와 보너스 번호가 일치한 2등은 66명으로 각 5653만9361원씩을, 당첨번호 5개를 맞힌 3등은 2636명으로 141만5629원씩을 받는다.당첨번호 4개를 맞힌 4등(고정 당첨금 5만원)은 12만9473명, 당첨번호 3개가 일치한 5등(고정 당첨금 5000원)은 214만9319명이다.이재덕 기자 duk@kyunghyang.com',)


In [ ]:
# df = pd.read_excel('./sample.xls', names=['title', 'press', 'category', 'date', 'name', 'text']) # sample.xls 헤더 이름을 붙여 가져오기
# txt = ''
# for i in df.index:
#     txt = txt +' ' +str(df['title'].loc[i]) # 기사 제목을 연달아 출력 ex) '기사1 제목' + '기사2 제목' + '기사3 제목' -> 고쳐야 되는 부분(기사 내용이 들어가고 기사 하나씩 들어가게 변경해야함)
    
# print(txt)

 중국에 대립각 세우는 리투아니아…왜? 국민의힘, 계속 옮겨붙는 갈등의 불씨...진화할 수 있을까 [단독] 법관이 직원에게 성경 공부·의전 지시, 법원에서도 ‘직장 갑질’ 논란 코로나 확진자수 2000명 돌파에 노심초사 여, 공세 강화하는 야 류호정 "피해자 보호 못하는 강간죄 개정 위해 '쇼'라도 하겠다" [뉴스분석]현실화된 정은경의 예언···확진자 2223명이 정점은 아니다 [비동의강간죄 발의 1년]이 법은 또 사라지는 중입니까? 윤석열, 통신선 복원 '이면협의' 의혹제기, 청와대 "사실 아니다" 김여정 '주한미군 철수 주장' 담화의 의미는 인건비 늘고 신작 출시 지연···게임 '3N', 나란히 2분기 실적부진 2024년 '달 착륙 계획’ 지연 되나…"신형 우주복 개발 늦어져" [인터랙티브] “이 법은 또 사라지는 중입니까”…비동의강간죄 발의 1년, 여전히 계류중 더 가까워지는 중·러, 고비사막에서 사흘째 합동군사훈련 ‘호프 알바’ 나선 정의당 대표…현장 행보로 ‘차별화’하면서 대선 준비 시동 코로나19 재확산에 온라인 유통망 발달까지…창신동 문구거리의 ‘깊어지는 시름’ 최재형 “여당은 청와대 출장소···비서관들이 국정 쥐락펴락” 문 대통령 성토 온난화에 '통제 불능' 미국·유럽 산불…미래의 예고편? “집단면역 불가능”…영국·싱가포르의 '코로나와 함께 살기' 실험 모더나 실무협의 위해 이번주 출국하는 정부대표단, 수급 차질 해법 있나 문 대통령, 국회에 고승범·송두환 후보자 인사청문 요청 국민의힘, 대선 후보 토론만 20회···흥행 묘수될까, 분란 씨앗될까 이재용 가석방에 ‘침묵’하는 문 대통령…여당서도 “가석방 이유 궁색” 황의탁 전북도의원 의정비 전액 복지시설에 ‘통 큰’ 기부 [경향포토] 경실련 이재용 가석방 반대 1인시위 ‘명·낙 대전’ 꺾이지 않는 상호 비방전…당 내선 “지도부가 중심잡아야” 소형 SUV 2강 자리 굳힌 트레일블레이저, 프리미엄 전략 통했다 가상자산 거래소 '불공정 약관' 손질에도 소비자 마음 못 놓는 이유는? 연상의 아내 마지막 

In [6]:
!curl -O https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5064  100  5064    0     0  37235      0 --:--:-- --:--:-- --:--:-- 37235


In [7]:
!bash ./mecab.sh

Installing automake (A dependency for mecab-ko)
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,263 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelea

In [29]:
from konlpy.tag import Mecab

In [51]:
mecab = Mecab() # mecab 사용

In [52]:
# 의미 없는 단어(조사 등) 제외
stopwords = ['은', '도', '는', '다', '의', '가', '이', '했', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '것', '에서', '이고', '라고', '다고', '라기', '라며', '면서', '라면서', '로써', '로서', '로', '으로', '뿐', '에서', '어야', '부터', '었', '았', '한다', '이다', '였다', '였었다', ',' , '.', '>', '<', ')', '(']

In [53]:
def mec(i):
  words = list()
  for tok in articles[i]:
    encoded = mecab.morphs(tok)
    wd = [item for item in encoded if item not in stopwords]
    words.append(encoded)

  return words

In [54]:
print(mec(1))

[['김학범', '감독', '이', '이끄', '는', '한국', '올림픽', '축구', '대표', '팀', '이', '31', '일', '일본', '요코하마', '스타디움', '에서', '열린', '2020', '도쿄', '올림픽', '남자', '축구', '8', '강', '멕시코', '와', '의', '경기', '에서', '3', '-', '6', '의', '굴욕', '적', '인', '완패', '를', '당했', '다', '.', '한국', '은', '시작', '하', '자마자', '멕시코', '에게', '선제골', '을', '내줬', '다', '.', '전반', '12', '분', '왼쪽', '에서', '올라온', '크로스', '를', '루이스', '로모', '가', '헤딩', '으로', '떨궜', '고', ',', '문전', '앞', '에', '있', '던', '엔리', '마르틴', '이', '골', '로', '연결', '시켰', '다', '.', '반격', '에', '나선', '한국', '은', '곧바로', '동점', '골', '을', '터뜨렸', '다', '.', '전반', '20', '분', '이동경', '이', '김진규', '의', '패스', '를', '받', '아', '수비수', '를', '한', '명', '제친', '뒤', '날카로운', '왼발', '슈팅', '으로', '골', '을', '성공', '시켰', '다', '.', '조금', '씩', '공세', '를', '이', '어', '가', '던', '한국', '은', '전반', '30', '분', '일격', '을', '맞', '고', '다시', '분위기', '가', '꺾였', '다', '.', '문전', '으로', '침투', '하', '던', '루이스', '로모', '가', '하프', '라인', '왼쪽', '에서', '올라온', '롱', '패스', '를', '잡', '아', '골', '로', '연결', '했', '다', '.', '한국', '은', '이어', '전반', '39', '분', 

In [39]:
for tok in articles:
  encoded = mecab.morphs(tok)
  sente = [item for item in encoded if item not in stopwords]
  articles.append(sente)

NotImplementedError: ignored

In [33]:
articles[0].str.replace('[^가-힣ㄱ-ㅎㅠ ]','')  # 해당 글자 외 다 제외

AttributeError: ignored

In [ ]:
!python -m pip install konlpy

In [ ]:
from konlpy.tag import Okt
okt = Okt()
morp = okt.morphs(txt)
print(morp)

['중국', '에', '대립', '각', '세우는', '리투아니아', '…', '왜', '?', '국민', '의', '힘', ',', '계속', '옮겨', '붙는', '갈등', '의', '불씨', '...', '진화', '할', '수', '있을까', '[', '단독', ']', '법관', '이', '직원', '에게', '성경', '공부', '·', '의전', '지시', ',', '법원', '에서도', '‘', '직장', '갑질', '’', '논란', '코로나', '확진', '자수', '2000', '명', '돌파', '에', '노심초사', '여', ',', '공세', '강화하는', '야', '류', '호정', '"', '피해자', '보호', '못', '하는', '강간죄', '개정', '위해', "'", '쇼', "'", '라도', '하겠다', '"', '[', '뉴스', '분석', ']', '현실', '화', '된', '정은경', '의', '예언', '···', '확', '진자', '2223', '명', '이', '정점', '은', '아니다', '[', '비', '동의', '강간죄', '발의', '1년', ']', '이', '법', '은', '또', '사라지는', '중입', '니까', '?', '윤석열', ',', '통신', '선', '복원', "'", '이면', '협의', "'", '의혹', '제기', ',', '청와대', '"', '사실', '아니다', '"', '김여정', "'", '주', '한', '미군', '철수', '주장', "'", '담화', '의', '의미', '는', '인건비', '늘', '고', '신작', '출시', '지연', '···', '게임', "'", '3', 'N', "',", '나란히', '2분', '기', '실적', '부진', '2024년', "'", '달', '착륙', '계획', '’', '지연', '되나', '…"', '신형', '우주복', '개발', '늦어져', '"', '[', '인터', '랙티브', ']', '“', '이

#### 사전에 단어 추가하는 부분(단어 추가하지 않을꺼면 필요없음)

In [ ]:
SentiWord = pd.read_csv('SentiWord_info.csv') # csv를 불러옴
word = '알쏭'
SentiWord.loc[len(SentiWord)] = ['징용','징용',-2]  # 맨 끝에 징용 -2를 줌
SentiWord.loc[len(SentiWord)] = ['합의','합의',1] # 맨 끝에 합의 +1 을 줌
SentiWord.tail()

def pos_neg(word):
    tmp =SentiWord[(SentiWord['word']==word) | (SentiWord['word_root']==word)]
    try:
        word_res = (word,tmp['polarity'][tmp.index[0]])
    except:
        word_res = (word, 0)
    return word_res

pos_neg('알쏭')

('알쏭', 0)

In [ ]:
from tqdm import tqdm
pos_list = []
neg_list = []
unkown_list = []

for noun in tqdm(morp, '긍/부정 나누기'):
    word_res = pos_neg(noun)
    if word_res[1] > 0:
        pos_list.append(word_res[0])
    elif word_res[1] < 0 :
        neg_list.append(word_res[0])
    else:
        unkown_list.append(word_res)

긍/부정 나누기: 100%|██████████| 1570/1570 [00:04<00:00, 348.80it/s]


In [ ]:
print('긍정키워드 수:', len(pos_list),'개')
print('부정키워드 수:', len(neg_list),'개')
print('중립 또는 모름 키워드 수:', len(unkown_list),'개')

긍정키워드 수: 61 개
부정키워드 수: 114 개
중립 또는 모름 키워드 수: 1395 개


In [ ]:
from collections import Counter #jdk 제공 모듈

pos_count = dict(Counter(pos_list).most_common()) # 단어별 갯수 계산 및 내림차순 정렬, 딕셔너리 구조
neg_count = dict(Counter(neg_list).most_common())
unkown_count = dict(Counter(unkown_list).most_common())
print(pos_count)

{'로': 8, '명': 6, '원': 4, '개': 4, '심': 3, '동의': 2, '와': 2, '당': 2, '성장': 2, '용기': 2, '것': 2, '위': 2, '보호': 1, '발달': 1, '함께': 1, '흥행': 1, '기부': 1, '(': 1, '인정': 1, '강화': 1, '대상': 1, '덕': 1, '수익': 1, '필요': 1, '가능': 1, '충족': 1, '정의': 1, '발전': 1, '상승세': 1, '적극': 1, '기대': 1, '신': 1, '세': 1, '중요한': 1}


In [ ]:
stop_words = ['은', '도', '는', '다', '의', '가', '이', '했', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게', '것', '었', ',' , '.', '>', '<', ')', '(', '에서', '이고', '라고', '다고', '라며', '면서', '라면서', '로써', '로서','로','으로','뿐', '에서','부터' ]

In [ ]:
tmp_dic = {}
for key, value in pos_count.items():
    if key not in stop_words:
        tmp_dic[key]=value

pos_count = tmp_dic

print(tmp_dic)

{'명': 6, '원': 4, '개': 4, '심': 3, '동의': 2, '당': 2, '성장': 2, '용기': 2, '위': 2, '보호': 1, '발달': 1, '함께': 1, '흥행': 1, '기부': 1, '인정': 1, '강화': 1, '대상': 1, '덕': 1, '수익': 1, '필요': 1, '가능': 1, '충족': 1, '정의': 1, '발전': 1, '상승세': 1, '적극': 1, '기대': 1, '신': 1, '세': 1, '중요한': 1}


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
### https://www.w3schools.com/colors/colors_hexadecimal.asp
def color_func(**kawargs):
    color="#0b1177"
    return color

In [ ]:
img_path = 'good.jpg'
cloud_mask = np.array(Image.open(img_path))
cloud_mask.shape

img = Image.open(img_path)
img.size

(831, 789)

In [ ]:
wordcloud = WordCloud(font_path="c:/Windows/Fonts/malgun.ttf",
                      background_color="white", mask=cloud_mask)
wc = wordcloud.generate_from_frequencies(pos_count)
plt.figure(figsize=(10,15))
plt.imshow(wc.recolor(color_func=color_func), interpolation='bilinear')
plt.axis("off")
plt.show()

OSError: ignored

### 감성사전에 긍정단어, 부정단어 추가하는 부분이라고 생각함

In [ ]:
SentiWord = pd.read_csv('SentiWord_info.csv') # csv를 불러옴
word = '알쏭'
SentiWord.loc[len(SentiWord)] = ['징용','징용',-2]  # 맨 끝에 징용 -2를 줌
SentiWord.loc[len(SentiWord)] = ['합의','합의',1] # 맨 끝에 합의 +1 을 줌
SentiWord.tail()

def pos_neg(word):
    tmp =SentiWord[(SentiWord['word']==word) | (SentiWord['word_root']==word)]
    try:
        word_res = (word,tmp['polarity'][tmp.index[0]])
    except:
        word_res = (word, 0)
    return word_res

pos_neg('알쏭')